Grok Assisted Version


Wolfrank Guzman
GhostOkaamiii

In [1]:
# AlgoHaus Backtester v5.1 - CustomTkinter Dark Edition (PARQUET VERSION - FINAL WORKING)

import customtkinter as ctk
import tkinter as tk
from tkinter import messagebox, filedialog
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import webbrowser
import os
import tempfile
import inspect
import pathlib
import threading
import logging
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("dark-blue")
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# ======================================================================
# 1. FOREX CALCULATOR
# ======================================================================
class ForexCalculator:
    LEVERAGE_OPTIONS = [1, 10, 20, 30, 50, 100, 200, 500]

    PIP_VALUES = {
        'EUR/USD': 0.0001, 'GBP/USD': 0.0001, 'USD/JPY': 0.01,
        'USD/CHF': 0.0001, 'USD/CAD': 0.0001, 'AUD/USD': 0.0001,
        'NZD/USD': 0.0001, 'EUR/JPY': 0.01, 'GBP/JPY': 0.01,
        'AUD/JPY': 0.01, 'EUR/GBP': 0.0001, 'EUR/CHF': 0.0001,
        'GBP/CHF': 0.0001, 'CHF/JPY': 0.01, 'CAD/JPY': 0.01
    }

    USD_MAJORS = {'EUR/USD', 'GBP/USD', 'AUD/USD', 'NZD/USD', 'USD/JPY', 'USD/CHF', 'USD/CAD'}

    @staticmethod
    def is_usd_major(pair):
        return pair in ForexCalculator.USD_MAJORS

    @staticmethod
    def calculate_pip_value_in_usd(pair, unit_size, current_price, conversion_rate=1.0):
        pip_size = ForexCalculator.PIP_VALUES.get(pair, 0.0001)
        if pair.endswith('/USD'):
            return pip_size * unit_size
        elif pair.startswith('USD/'):
            return (pip_size / current_price) * unit_size
        else:
            return pip_size * unit_size * conversion_rate

    @staticmethod
    def calculate_margin_required(pair, unit_size, current_price, leverage, conversion_rate=1.0):
        if pair.endswith('/USD'):
            value = unit_size * current_price
        elif pair.startswith('USD/'):
            value = unit_size
        else:
            value = unit_size * conversion_rate
        return value / leverage

    @staticmethod
    def calculate_position_size(balance, risk_pct, sl_pips, pair, price, conversion_rate=1.0):
        risk_amount = balance * (risk_pct / 100)
        pip_val_per_unit = ForexCalculator.calculate_pip_value_in_usd(pair, 1, price, conversion_rate)
        if pip_val_per_unit <= 0:
            return 0
        size = risk_amount / (sl_pips * pip_val_per_unit)
        return max(1000, int(round(size / 1000)) * 1000)

# ======================================================================
# 2. DATA LOADING
# ======================================================================
def detect_available_pairs(base_folder: pathlib.Path):
    pairs = set()
    for file in base_folder.rglob("*.parquet"):
        name = file.stem.replace('_', '/')
        if '/' in name and len(name.split('/')) == 2:
            pairs.add(name)
    return sorted(list(pairs))

def load_pair_data(pair_name: str, base_folder: pathlib.Path, start_date: datetime, end_date: datetime, timeframe: str):
    pair_clean = pair_name.replace('/', '_')
    subfolder = base_folder / pair_clean
    parquet_files = list(subfolder.glob("*.parquet")) if subfolder.is_dir() else []
    if not parquet_files:
        parquet_files = [f for f in base_folder.glob("*.parquet") if pair_clean.lower() in f.name.lower()]
    if not parquet_files:
        raise FileNotFoundError(f"No PARQUET file found for {pair_name}")
    df = pd.read_parquet(parquet_files[0], engine='pyarrow')
    if df.empty:
        raise ValueError("PARQUET file is empty")

    cols_lower = [c.strip().lower() for c in df.columns]
    col_map = {
        'datetime': ['datetime', 'date', 'time', 'timestamp', 'date_time', 'index'],
        'open': ['open', 'o'],
        'high': ['high', 'h'],
        'low': ['low', 'l'],
        'close': ['close', 'c', 'last'],
        'volume': ['volume', 'vol', 'v']
    }
    rename = {}
    for target, aliases in col_map.items():
        for alias in aliases:
            if any(alias in col for col in cols_lower):
                orig = next(col for col in df.columns if alias in col.lower())
                rename[orig] = target
                break
        else:
            if target != 'volume':
                raise KeyError(f"Column for '{target}' not found")
    df = df.rename(columns=rename)
    if 'volume' not in df.columns:
        df['volume'] = 1000

    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', utc=True)
    df = df.dropna(subset=['datetime']).drop_duplicates(subset='datetime').sort_values('datetime')
    df['datetime'] = df['datetime'].dt.tz_localize(None)

    df = df.set_index('datetime')
    user_start = max(pd.Timestamp(start_date.date()), df.index.min())
    user_end = min(pd.Timestamp(end_date.date()) + pd.Timedelta(hours=23, minutes=59), df.index.max())
    df = df.loc[user_start:user_end].copy()
    if df.empty:
        raise ValueError("No data in requested range")

    if timeframe != '1min':
        rule = {'5min': '5T', '15min': '15T', '1hr': '1H', '1Day': '1D'}.get(timeframe, '1T')
        df = df.resample(rule).agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'}).dropna()

    df = df.reset_index()
    df['date'] = df['datetime'].dt.date
    daily = df.groupby('date').agg({'high': 'max', 'low': 'min', 'close': 'last'})
    daily['prev_high'] = daily['high'].shift(1)
    daily['prev_low'] = daily['low'].shift(1)
    daily['prev_close'] = daily['close'].shift(1)
    df = df.merge(daily[['prev_high', 'prev_low', 'prev_close']], left_on='date', right_index=True, how='left')
    df[['prev_high', 'prev_low', 'prev_close']] = df[['prev_high', 'prev_low', 'prev_close']].ffill()
    return df

# ======================================================================
# 3. STRATEGIES
# ======================================================================
class TradingStrategies:
    @staticmethod
    def opening_range_strategy(df, sl_pips, tp_pips, pip_value):
        trades = []
        for date_val in df['date'].unique():
            day_data = df[df['date'] == date_val].reset_index(drop=True)
            if len(day_data) < 31:
                continue
            or_high = day_data.iloc[:30]['high'].max()
            or_low = day_data.iloc[:30]['low'].min()
            or_mid = (or_high + or_low) / 2
            if len(day_data) > 30:
                entry_bar = day_data.iloc[30]
                signal = 'BUY' if entry_bar['close'] > or_mid else 'SELL'
                trades.append({
                    'datetime': entry_bar['datetime'],
                    'entry_price': entry_bar['close'],
                    'signal': signal,
                    'day_data': day_data.iloc[31:].reset_index(drop=True)
                })
        return trades

    @staticmethod
    def vwap_crossover_strategy(df, sl_pips, tp_pips, pip_value):
        df = df.copy()
        df['tpv'] = df['volume'] * (df['high'] + df['low'] + df['close']) / 3
        df['cumvol'] = df.groupby('date')['volume'].cumsum()
        df['cumtpv'] = df.groupby('date')['tpv'].cumsum()
        df['vwap'] = df['cumtpv'] / df['cumvol']
        df['prev_close'] = df['close'].shift(1)
        df['prev_vwap'] = df['vwap'].shift(1)
        df['signal'] = np.where(
            (df['prev_close'] <= df['prev_vwap']) & (df['close'] > df['vwap']), 'BUY',
            np.where((df['prev_close'] >= df['prev_vwap']) & (df['close'] < df['vwap']), 'SELL', np.nan)
        )
        entries = df.dropna(subset=['signal'])
        trades = []
        for _, row in entries.iterrows():
            remaining = df[df['datetime'] > row['datetime']].copy()
            if not remaining.empty:
                trades.append({
                    'datetime': row['datetime'],
                    'entry_price': row['close'],
                    'signal': row['signal'],
                    'day_data': remaining
                })
        return trades

    @staticmethod
    def pivot_point_reversal_strategy(df, sl_pips, tp_pips, pip_value):
        df = df.copy()
        df['pivot'] = (df['prev_high'] + df['prev_low'] + df['prev_close']) / 3
        df['r1'] = 2 * df['pivot'] - df['prev_low']
        df['s1'] = 2 * df['pivot'] - df['prev_high']
        df = df[df['prev_high'].notna()].copy()
        bounce_zone = 5 * pip_value
        trades = []
        for i in range(len(df)):
            row = df.iloc[i]
            if abs(row['close'] - row['s1']) < bounce_zone:
                remaining = df.iloc[i+1:]
                if not remaining.empty:
                    trades.append({'datetime': row['datetime'], 'entry_price': row['close'], 'signal': 'BUY', 'day_data': remaining})
            elif abs(row['close'] - row['r1']) < bounce_zone:
                remaining = df.iloc[i+1:]
                if not remaining.empty:
                    trades.append({'datetime': row['datetime'], 'entry_price': row['close'], 'signal': 'SELL', 'day_data': remaining})
        return trades

# ======================================================================
# 4. OPTIMIZED BACKTESTER
# ======================================================================
class EnhancedBacktester:
    def __init__(self, df, initial_balance=10000, pip_value=0.0001, leverage=50, risk_percent=1.0):
        self.df = df
        self.initial_balance = initial_balance
        self.pip_value = pip_value
        self.leverage = leverage
        self.risk_percent = risk_percent

    def run_backtest(self, strategy_func, sl_pips, tp_pips, pair_name):
        trades = strategy_func(self.df, sl_pips, tp_pips, self.pip_value)
        if not trades:
            return "No trades generated.", pd.DataFrame()

        results = []
        balance = self.initial_balance
        trade_num = 1

        is_usd_major = ForexCalculator.is_usd_major(pair_name)

        for trade in trades:
            entry_time = trade['datetime']
            entry_price = trade['entry_price']
            signal = trade['signal']
            bars = trade['day_data']  # FIXED: removed the dot
            if bars.empty:
                continue

            unit_size = ForexCalculator.calculate_position_size(balance, self.risk_percent, sl_pips, pair_name, entry_price)
            if unit_size < 1000:
                continue

            margin = ForexCalculator.calculate_margin_required(pair_name, unit_size, entry_price, self.leverage)
            if margin > balance * 0.8:
                continue

            pip_val_usd = ForexCalculator.calculate_pip_value_in_usd(pair_name, unit_size, entry_price)

            sl_level = entry_price - sl_pips * self.pip_value if signal == 'BUY' else entry_price + sl_pips * self.pip_value
            tp_level = entry_price + tp_pips * self.pip_value if signal == 'BUY' else entry_price - tp_pips * self.pip_value

            if signal == 'BUY':
                sl_hit = bars['low'] <= sl_level
                tp_hit = bars['high'] >= tp_level
            else:
                sl_hit = bars['high'] >= sl_level
                tp_hit = bars['low'] <= tp_level

            hit_idx = np.where(sl_hit | tp_hit)[0]
            if hit_idx.size > 0:
                idx = hit_idx[0]
                exit_price = sl_level if sl_hit.iloc[idx] else tp_level
                reason = 'SL' if sl_hit.iloc[idx] else 'TP'
            else:
                idx = len(bars) - 1
                exit_price = bars.iloc[idx]['close']
                reason = 'Timeout'

            exit_time = bars.iloc[idx]['datetime']
            hours = (exit_time - entry_time).total_seconds() / 3600
            pips = (exit_price - entry_price) / self.pip_value if signal == 'BUY' else (entry_price - exit_price) / self.pip_value
            pnl = pips * pip_val_usd
            balance += pnl

            results.append({
                'trade_number': f"{trade_num:05d}",
                'entry_time': entry_time,
                'exit_time': exit_time,
                'time_in_trade_hours': round(hours, 2),
                'signal': signal,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'exit_reason': reason,
                'pips_pnl': round(pips, 2),
                'monetary_pnl': round(pnl, 2),
                'unit_size': unit_size,
                'margin_used': round(margin, 2),
                'balance': round(balance, 2)
            })
            trade_num += 1

        results_df = pd.DataFrame(results)
        total_pnl = results_df['monetary_pnl'].sum() if not results_df.empty else 0
        total_pips = results_df['pips_pnl'].sum() if not results_df.empty else 0
        win_rate = (results_df['pips_pnl'] > 0).mean() * 100 if not results_df.empty else 0
        summary = f"TRADES: {len(results_df)}\nWIN RATE: {win_rate:.1f}%\nP&L: ${total_pnl:,.2f}\nPIPS: {total_pips:,.1f}\nFINAL BALANCE: ${balance:,.2f}"
        if not is_usd_major:
            summary += "\nWarning: Cross pair - Pip values approximate"
        return summary, results_df

# ======================================================================
# 5. HTML REPORT GENERATOR
# ======================================================================
class HTMLReportGenerator:
    @staticmethod
    def generate_report(trades_df, summary_text, pair, strategy_name, timeframe, initial_balance, df):
        if trades_df.empty or df is None or df.empty:
            return None

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        equity = initial_balance + trades_df['monetary_pnl'].cumsum()

        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.7, 0.3])
        fig.add_trace(go.Candlestick(x=df['datetime'],
                                     open=df['open'], high=df['high'],
                                     low=df['low'], close=df['close'],
                                     name="Price"), row=1, col=1)
        fig.add_trace(go.Scatter(x=trades_df['exit_time'], y=equity,
                                 mode='lines', name='Equity Curve',
                                 line=dict(color='#00ff88', width=2)), row=2, col=1)
        fig.update_layout(height=700, template="plotly_dark",
                          title_text=f"{pair} - {strategy_name} - {timeframe}")
        equity_html = fig.to_html(full_html=False, include_plotlyjs='cdn')

        html = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>AlgoHaus Report - {pair} {strategy_name}</title>
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
            <style>
                body {{ font-family: Arial, sans-serif; background: #000; color: #fff; padding: 20px; }}
                .container {{ max-width: 1400px; margin: auto; background: #111; padding: 30px; border-radius: 10px; }}
                .header {{ text-align: center; padding: 30px; background: #1976D2; border-radius: 10px; margin-bottom: 20px; }}
                pre {{ background: #222; padding: 15px; border-radius: 8px; overflow-x: auto; }}
                table {{ width: 100%; border-collapse: collapse; margin: 20px 0; }}
                th, td {{ border: 1px solid #333; padding: 10px; text-align: center; }}
                th {{ background: #1976D2; }}
                .win {{ color: #4CAF50; font-weight: bold; }}
                .loss {{ color: #F44336; font-weight: bold; }}
            </style>
        </head>
        <body>
        <div class="container">
            <div class="header">
                <h1>AlgoHaus Backtest Report</h1>
                <p>{pair} | {strategy_name} | {timeframe} | {timestamp}</p>
            </div>
            <pre>{summary_text}</pre>
            <div>{equity_html}</div>
            <h2>Detailed Trade Log</h2>
            <table>
                <tr>
                    <th>#</th><th>Entry Time</th><th>Exit Time</th><th>Signal</th>
                    <th>Entry Price</th><th>Exit Price</th><th>P&L ($)</th><th>Pips</th><th>Reason</th>
                </tr>
        """
        for _, row in trades_df.iterrows():
            pnl_class = "win" if row['monetary_pnl'] > 0 else "loss"
            html += f"""
                <tr>
                    <td>{row['trade_number']}</td>
                    <td>{row['entry_time'].strftime('%Y-%m-%d %H:%M')}</td>
                    <td>{row['exit_time'].strftime('%Y-%m-%d %H:%M')}</td>
                    <td>{row['signal']}</td>
                    <td>{row['entry_price']:.5f}</td>
                    <td>{row['exit_price']:.5f}</td>
                    <td class="{pnl_class}">${row['monetary_pnl']:+,.2f}</td>
                    <td class="{pnl_class}">{row['pips_pnl']:+.1f}</td>
                    <td>{row['exit_reason']}</td>
                </tr>
            """
        html += """
            </table>
        </div>
        </body>
        </html>
        """

        path = os.path.join(tempfile.gettempdir(), f"AlgoHaus_Report_{pair.replace('/', '_')}_{timestamp}.html")
        with open(path, "w", encoding="utf-8") as f:
            f.write(html)
        return path

# ======================================================================
# 6. UI
# ======================================================================
class BacktesterUI:
    def __init__(self, master):
        self.master = master
        master.title("AlgoHaus Backtester v5.1 - PARQUET Edition")
        master.geometry("1400x900")
        master.minsize(1200, 700)

        self.data_folder = pathlib.Path.cwd() / "data"
        self.df = None
        self.trades_df = pd.DataFrame()

        self.selected_pair = tk.StringVar(master, value="EUR/USD")
        self.selected_timeframe = tk.StringVar(master, value="1hr")
        self.selected_strategy = tk.StringVar(master, value="vwap_crossover_strategy")
        self.initial_balance = tk.DoubleVar(master, value=10000.0)
        self.leverage = tk.IntVar(master, value=50)
        self.sl_pips = tk.IntVar(master, value=30)
        self.tp_pips = tk.IntVar(master, value=60)
        self.risk_percent = tk.DoubleVar(master, value=1.0)

        today = date.today()
        self.end_date_var = tk.StringVar(master, value=today.strftime("%Y-%m-%d"))
        self.start_date_var = tk.StringVar(master, value=(today - timedelta(days=365)).strftime("%Y-%m-%d"))

        self.status_text = tk.StringVar(master, value="Ready.")

        self.setup_ui()
        self.refresh_available_pairs()

    def setup_ui(self):
        main_container = ctk.CTkFrame(self.master)
        main_container.pack(fill="both", expand=True)

        left_panel = ctk.CTkFrame(main_container, width=450)
        left_panel.pack(side="left", fill="both", padx=20, pady=20)
        left_panel.pack_propagate(False)

        right_panel = ctk.CTkFrame(main_container)
        right_panel.pack(side="right", fill="both", expand=True, padx=20, pady=20)

        # Left Panel
        title = ctk.CTkLabel(left_panel, text="AlgoHaus Backtester v5.1", font=ctk.CTkFont(size=18, weight="bold"))
        title.pack(pady=20)

        scroll = ctk.CTkScrollableFrame(left_panel)
        scroll.pack(fill="both", expand=True)

        config = ctk.CTkFrame(scroll)
        config.pack(fill="x", pady=10)
        ctk.CTkLabel(config, text="Configuration", font=ctk.CTkFont(weight="bold")).pack(anchor="w", padx=15, pady=5)

        folder_frame = ctk.CTkFrame(config)
        folder_frame.pack(fill="x", padx=15, pady=5)
        ctk.CTkLabel(folder_frame, text="Data Folder:").pack(side="left")
        self.folder_label = ctk.CTkLabel(folder_frame, text="data/")
        self.folder_label.pack(side="left", fill="x", expand=True, padx=10)
        ctk.CTkButton(folder_frame, text="Browse", command=self.select_data_folder).pack(side="right")

        self.create_input_field(config, "Trading Pair:", self.selected_pair, is_combobox=True)
        self.create_input_field(config, "Timeframe:", self.selected_timeframe, is_combobox=True, values=["1min", "5min", "15min", "1hr", "1Day"])
        self.create_input_field(config, "Start Date:", self.start_date_var)
        self.create_input_field(config, "End Date:", self.end_date_var)

        strat = ctk.CTkFrame(scroll)
        strat.pack(fill="x", pady=10)
        ctk.CTkLabel(strat, text="Strategy & Risk", font=ctk.CTkFont(weight="bold")).pack(anchor="w", padx=15, pady=5)
        strategies = [name for name, obj in inspect.getmembers(TradingStrategies) if inspect.isfunction(obj)]
        self.create_input_field(strat, "Strategy:", self.selected_strategy, is_combobox=True, values=strategies)
        self.create_input_field(strat, "Stop Loss (Pips):", self.sl_pips)
        self.create_input_field(strat, "Take Profit (Pips):", self.tp_pips)

        acc = ctk.CTkFrame(scroll)
        acc.pack(fill="x", pady=10)
        ctk.CTkLabel(acc, text="Account Settings", font=ctk.CTkFont(weight="bold")).pack(anchor="w", padx=15, pady=5)
        self.create_input_field(acc, "Initial Balance ($):", self.initial_balance)
        self.create_input_field(acc, "Risk % per Trade:", self.risk_percent)
        self.create_input_field(acc, "Leverage:", self.leverage, is_combobox=True, values=[str(x) for x in ForexCalculator.LEVERAGE_OPTIONS])

        ctk.CTkButton(left_panel, text="RUN BACKTEST", command=self.start_backtest_thread,
                      font=ctk.CTkFont(size=14, weight="bold"), height=50).pack(fill="x", padx=20, pady=20)

        # Right Panel
        summary_frame = ctk.CTkFrame(right_panel)
        summary_frame.pack(fill="x", pady=10)
        ctk.CTkLabel(summary_frame, text="Summary", font=ctk.CTkFont(weight="bold")).pack(anchor="w", padx=15, pady=5)
        self.summary_textbox = ctk.CTkTextbox(summary_frame, height=150)
        self.summary_textbox.pack(fill="both", expand=True, padx=15, pady=5)

        metrics_frame = ctk.CTkFrame(right_panel)
        metrics_frame.pack(fill="both", expand=True, pady=10)
        ctk.CTkLabel(metrics_frame, text="Metrics", font=ctk.CTkFont(weight="bold")).pack(anchor="w", padx=15, pady=5)
        self.metrics_scroll = ctk.CTkScrollableFrame(metrics_frame)
        self.metrics_scroll.pack(fill="both", expand=True, padx=15, pady=5)

        self.report_button = ctk.CTkButton(right_panel, text="Generate HTML Report",
                                           command=self.generate_html_report, state="disabled")
        self.report_button.pack(fill="x", pady=15)

        status = ctk.CTkFrame(self.master, height=35)
        status.pack(side="bottom", fill="x")
        ctk.CTkLabel(status, textvariable=self.status_text).pack(side="left", padx=20, pady=8)

    def create_input_field(self, parent, label_text, variable, is_combobox=False, values=None):
        frame = ctk.CTkFrame(parent)
        frame.pack(fill="x", padx=15, pady=5)
        ctk.CTkLabel(frame, text=label_text).pack(side="left")
        if is_combobox:
            widget = ctk.CTkComboBox(frame, variable=variable, values=values or [])
        else:
            widget = ctk.CTkEntry(frame, textvariable=variable)
        widget.pack(side="right", fill="x", expand=True, padx=10)
        return widget

    def update_status(self, text):
        self.status_text.set(text)

    def refresh_available_pairs(self):
        pairs = detect_available_pairs(self.data_folder)
        if pairs:
            for widget in self.master.winfo_children()[0].winfo_children()[0].winfo_children():
                if isinstance(widget, ctk.CTkFrame) and widget.winfo_children():
                    if isinstance(widget.winfo_children()[1], ctk.CTkComboBox) and widget.winfo_children()[0].cget("text") == "Trading Pair:":
                        widget.winfo_children()[1].configure(values=pairs)
                        if self.selected_pair.get() not in pairs:
                            self.selected_pair.set(pairs[0])
                        break
            self.update_status(f"Found {len(pairs)} pairs")
        else:
            self.update_status("No data found")

    def select_data_folder(self):
        folder = filedialog.askdirectory()
        if folder:
            self.data_folder = pathlib.Path(folder)
            self.folder_label.configure(text=str(self.data_folder))
            self.refresh_available_pairs()

    def start_backtest_thread(self):
        self.update_status("Running backtest...")
        threading.Thread(target=self.run_backtest, daemon=True).start()

    def run_backtest(self):
        try:
            pair = self.selected_pair.get()
            timeframe = self.selected_timeframe.get()
            strategy_name = self.selected_strategy.get()
            start = datetime.strptime(self.start_date_var.get(), "%Y-%m-%d")
            end = datetime.strptime(self.end_date_var.get(), "%Y-%m-%d")
            df = load_pair_data(pair, self.data_folder, start, end, timeframe)
            self.df = df
            strategy_func = getattr(TradingStrategies, strategy_name)
            backtester = EnhancedBacktester(
                df,
                initial_balance=self.initial_balance.get(),
                pip_value=ForexCalculator.PIP_VALUES.get(pair, 0.0001),
                leverage=self.leverage.get(),
                risk_percent=self.risk_percent.get()
            )
            summary, results_df = backtester.run_backtest(strategy_func, self.sl_pips.get(), self.tp_pips.get(), pair)
            self.master.after(0, lambda s=summary, r=results_df: self.display_results(s, r))
        except Exception as e:
            self.master.after(0, lambda e=e: messagebox.showerror("Error", str(e)))
            self.master.after(0, lambda: self.update_status("Backtest failed"))

    def display_results(self, summary, trades_df):
        self.trades_df = trades_df
        self.summary_textbox.delete("1.0", "end")
        self.summary_textbox.insert("1.0", summary)
        self.report_button.configure(state="normal")
        self.update_status("Backtest complete - Report ready!")

    def generate_html_report(self):
        if self.trades_df.empty:
            messagebox.showwarning("No Trades", "No trades to include in the report.")
            return
        self.update_status("Generating HTML report...")
        try:
            report_path = HTMLReportGenerator.generate_report(
                self.trades_df,
                self.summary_textbox.get("1.0", "end"),
                self.selected_pair.get(),
                self.selected_strategy.get(),
                self.selected_timeframe.get(),
                self.initial_balance.get(),
                self.df
            )
            if report_path:
                webbrowser.open('file://' + os.path.realpath(report_path))
                self.update_status("HTML report generated and opened!")
            else:
                self.update_status("Failed to generate report.")
        except Exception as e:
            messagebox.showerror("Report Error", f"Failed to generate report:\n{e}")
            self.update_status("Report generation failed")

# ======================================================================
# MAIN
# ======================================================================
if __name__ == "__main__":
    app = ctk.CTk()
    ui = BacktesterUI(app)
    app.mainloop()

ModuleNotFoundError: No module named 'customtkinter'